Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [12]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [13]:
import funciones

In [14]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [15]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [16]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    cursor.execute("EXEC [dbo].[sp_get_precios_recolectados_mes] 2024, 1")
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    cursor.execute("EXEC sp_get_indice_grupo 2024, 2")
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [17]:
#boletas['cantidad_actual'].dtype
#boletas['cantidad_anterior'].dtype
#boletas['precio_anterior'].dtype
#boletas['precio_actual'].dtype
#boletas.dtypes

#ponderaciones['region_id'].dtype
#ponderaciones['ponderacion_region'].dtype
#ponderaciones['indice_anterior'].dtype
#ponderaciones.dtypes
#ponderaciones['indice_anterior'].isnull().sum()

In [18]:
boletas

,pk,region,anio,mes,decada,fuente_codigo,fuente_nombre,fuente_direccion,ine_poll_id,fuente_tipo,...,good_tp_status,poll_id,good_group_id,producto_nombre,good_technical_issue_id,nt_comentarios,nt_tipo,nt_tipo_nombre,nt_razon,nt_razon_nombre
0,249736,1,2024,1,Primera década,34,CRISTALERIA JEHOVA JIREN,INTERIOR MERCADO LA FLORIDA LOCAL 101 Zona : 1...,34-1-2024,Almacenes o tiendas especializadas,...,350049,47440,361,Ollas y cacerolas,NaN,None,NaN,,NaN,
1,249737,1,2024,1,Primera década,36,BOUTIQUE MEXICO LINDO,C.C. LA PULGA 5 AVENIDA 2-43 LOCAL 4 ( EL GUAR...,36-1-2024,Almacenes o tiendas especializadas,...,350049,47441,230,Vestidos formales y casuales para mujeres (may...,NaN,None,NaN,,NaN,
2,221637,1,2024,1,Primera década,42,AUTOLLANTAS Y SERVICIOS S.A ( KARINA DE LA ROCA),CALZADA AGUILAR BATRES 12-63 Zona : 12 GUATEMA...,42-1-2024,Almacenes o tiendas especializadas,...,350049,39716,495,Llantas nuevas,NaN,None,NaN,,NaN,
3,221638,1,2024,1,Primera década,43,ERICK SASO,"19 AVENIDA 31-72 Zona : 5 GUATEMALA, GUATEMALA",43-1-2024,Vivienda tipo apartamento de alquiler,...,350049,39717,284,Alquiler de vivienda,NaN,None,NaN,,NaN,
4,221641,1,2024,1,Primera década,54,ACEITERA TURIN / ODILIA BARRIENTOS,"AVENIDA PETAPA 14-54 Zona : 12 GUATEMALA, GUAT...",54-1-2024,Almacenes o tiendas especializadas,...,350049,39719,506,Aceite de motor,NaN,None,NaN,,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51522,281212,8,2024,1,Tercera década,18268,MAXI DESPENSA,"CALZADA GUATEMALA ZONA 5 BO. SANTA FE, POPTUN,...",18268-1-2024,"Supermercados, despensas y almacenes en cadena",...,350049,54461,359,Vajillas,NaN,None,NaN,,NaN,
51523,281213,8,2024,1,Tercera década,18268,MAXI DESPENSA,"CALZADA GUATEMALA ZONA 5 BO. SANTA FE, POPTUN,...",18268-1-2024,"Supermercados, despensas y almacenes en cadena",...,350049,54461,364,Vasos y tazas de plástico y melamina,NaN,None,NaN,,NaN,
51524,281214,8,2024,1,Tercera década,18268,MAXI DESPENSA,"CALZADA GUATEMALA ZONA 5 BO. SANTA FE, POPTUN,...",18268-1-2024,"Supermercados, despensas y almacenes en cadena",...,350049,54461,378,Jabón para ropa,NaN,None,NaN,,NaN,
51525,281215,8,2024,1,Tercera década,18268,MAXI DESPENSA,"CALZADA GUATEMALA ZONA 5 BO. SANTA FE, POPTUN,...",18268-1-2024,"Supermercados, despensas y almacenes en cadena",...,350049,54461,391,Velas y candelas,NaN,None,NaN,,NaN,


In [19]:
#esta parte la voy a usar para poder modificar la base de datos, luego la quito de este python
boletas01 = boletas[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']]
boletas01.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR'}, inplace=True)
boletas01.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)
boletas01.sort_index(inplace=True)
boletas01

C:\Users\sncap\AppData\Local\Temp\ipykernel_36824\3368766721.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boletas01.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR'}, inplace=True)
C:\Users\sncap\AppData\Local\Temp\ipykernel_36824\3368766721.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boletas01.sort_index(inplace=True)


UraChi UreCan   ArtPhi   ArtPac  ArtCR
RegCod ArtCod   BolNum                                             
1      10101011 1073-1-2024    1.00   1.00   300.00   300.00    NaN
                1209-1-2024    1.00   1.00  1873.50  1873.50    NaN
                1676-1-2024    1.00   1.00   750.00   750.00    NaN
                3198-1-2024    1.00   1.00   700.00   700.00    NaN
                3308-1-2024    1.00   1.00  1450.00  1450.00    NaN
...                             ...    ...      ...      ...    ...
8      9740083  16255-1-2024   1.00   1.00     2.00     2.00    NaN
                16326-1-2024   1.00   1.00     2.00     2.00    NaN
                16350-1-2024   1.00   1.00     2.00     2.00    NaN
                16485-1-2024   1.00   1.00     2.00     2.00    NaN
                20848-1-2024   1.00   1.00     2.00     2.00    NaN

[51527 rows x 5 columns]

In [53]:
recodificacion = {
    '1111011':'011110101', 
    '1111012':'011110103', #arroz
    '1111021':'011120201', 
    '1111022':'011120202', #maiz
    '1112011':'011120102', 
    '1112021':'011120101', 
    '1112022':'011120101', 
    '1112023':'011120101', #harina
    '1112031':'011120304', #preparacion nutricional
    '1113011':'011130101', #pan frances 
    '1113012':'011130103', #pan rodajado
    '1113021':'011130102', #pan dulce corriente
    '1113031':'011130201', #galletas
    '1113041':'011150101', 
    '1113042':'011150101',
    '1113043':'011150101', #tortillas frescas
    '1113051':'011130302', #magdalena
    '1114011':'011120301', #hojuelas de maiz
    '1114021':'011120305', 
    '1114022':'011120305', #mosh
    '1115011':'011140101', #espaguetti 
    '1115021':'011140103',
    '1115022':'011140103',
    '1115023':'011140103',
    '1115024':'011140103',
    '1115025':'011140103', #fideos 
    '1115021':'011140102', #chao mein
    '1122011':'011210101',
    '1122012':'011210102', 
    '1122013':'011210103',
    '1122014':'011210104',
    '1122015':'011210105',
    '1122016':'011210106',
    '1122017':'011210107', #carne de res
    '1122021':'011220101',
    '1122022':'011220102',
    '1122023':'011220104', #carne de cerdo
    '1122031':'011230101', #pollo 
    '1124021':'011210108', #higado de res
    '1125011':'011240101', 
    '1125012':'011240101', #salchichas
    '1125021':'011240104', #jamon de pavo
    '1125023':'011240102', #jamon de cerdo
    '1125031':'011240103', 
    '1125032':'011240103', #longaniza de cerdo
    '1125033':'011240105', 
    '1125034':'011240105', #chorizo colorados de cerdo
    '1125061':'011220103', #carne de cerdo adobada
    '1131011':'011310103',
    '1131012':'011310101', 
    '1131014':'011310102', #filete de pescado fresco
    '1133011':'011930503', 
    '1133012':'011930503', #sardinas
    '1134011':'011320101', 
    '1134012':'011320101', #camarones
    '1141012':'011410102', #leche liquida
    '1143011':'011410101',
    '1143012':'011410101', #leche entera en polvo
    '1143021':'011430103', #crema fresca pasteurizada
    '1145011':'011430101',
    '1145012':'011430101', #queso fresco
    '1145021':'011430104', #queso crema
    '1145041':'011430105',
    '1145042':'011430105', # queso seco
    '1146011':'011410201',
    '1146012':'011410201', #yogur preparado
    '1147012':'011410103', #leche liquida esterilizada
    '1148012':'011440101',
    '1148011':'011440101', #huevos de gallina
    '1151011':'011520101',
    '1151012':'011520101',
    '1151013':'011520101', #aceite corriente
    '1151014':'011520101', #aceite oliva
    '1153011':'011510101',
    '1153012':'011510101',
    '1161011':'011610101', #aguacate
    '1161021':'011610201', #bananos
    '1161031':'011610501', #platanos
    '1161041':'011610606', #mango
    #'':'011610601', #papaya
    #'':'011610603', #pina
    #'':'011610602', 
    #'':'011610602', #limon
    #'':'0',
    #'':'0',
    #'':'0',
    #'':'0',
    #'':'0',
    #'':'0',
    #'':'0',
    #'':'0',
}

In [54]:
test = boletas

In [55]:
test['codigo_articulo'] = test['codigo_articulo'].replace(recodificacion)
test.sort_index() #si funciona yay!!!! 

,pk,region,anio,mes,decada,fuente_codigo,fuente_nombre,fuente_direccion,ine_poll_id,fuente_tipo,...,good_tp_status,poll_id,good_group_id,producto_nombre,good_technical_issue_id,nt_comentarios,nt_tipo,nt_tipo_nombre,nt_razon,nt_razon_nombre
0,249736,1,2024,1,Primera década,34,CRISTALERIA JEHOVA JIREN,INTERIOR MERCADO LA FLORIDA LOCAL 101 Zona : 1...,34-1-2024,Almacenes o tiendas especializadas,...,350049,47440,361,Ollas y cacerolas,NaN,None,NaN,,NaN,
1,249737,1,2024,1,Primera década,36,BOUTIQUE MEXICO LINDO,C.C. LA PULGA 5 AVENIDA 2-43 LOCAL 4 ( EL GUAR...,36-1-2024,Almacenes o tiendas especializadas,...,350049,47441,230,Vestidos formales y casuales para mujeres (may...,NaN,None,NaN,,NaN,
2,221637,1,2024,1,Primera década,42,AUTOLLANTAS Y SERVICIOS S.A ( KARINA DE LA ROCA),CALZADA AGUILAR BATRES 12-63 Zona : 12 GUATEMA...,42-1-2024,Almacenes o tiendas especializadas,...,350049,39716,495,Llantas nuevas,NaN,None,NaN,,NaN,
3,221638,1,2024,1,Primera década,43,ERICK SASO,"19 AVENIDA 31-72 Zona : 5 GUATEMALA, GUATEMALA",43-1-2024,Vivienda tipo apartamento de alquiler,...,350049,39717,284,Alquiler de vivienda,NaN,None,NaN,,NaN,
4,221641,1,2024,1,Primera década,54,ACEITERA TURIN / ODILIA BARRIENTOS,"AVENIDA PETAPA 14-54 Zona : 12 GUATEMALA, GUAT...",54-1-2024,Almacenes o tiendas especializadas,...,350049,39719,506,Aceite de motor,NaN,None,NaN,,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51522,281212,8,2024,1,Tercera década,18268,MAXI DESPENSA,"CALZADA GUATEMALA ZONA 5 BO. SANTA FE, POPTUN,...",18268-1-2024,"Supermercados, despensas y almacenes en cadena",...,350049,54461,359,Vajillas,NaN,None,NaN,,NaN,
51523,281213,8,2024,1,Tercera década,18268,MAXI DESPENSA,"CALZADA GUATEMALA ZONA 5 BO. SANTA FE, POPTUN,...",18268-1-2024,"Supermercados, despensas y almacenes en cadena",...,350049,54461,364,Vasos y tazas de plástico y melamina,NaN,None,NaN,,NaN,
51524,281214,8,2024,1,Tercera década,18268,MAXI DESPENSA,"CALZADA GUATEMALA ZONA 5 BO. SANTA FE, POPTUN,...",18268-1-2024,"Supermercados, despensas y almacenes en cadena",...,350049,54461,378,Jabón para ropa,NaN,None,NaN,,NaN,
51525,281215,8,2024,1,Tercera década,18268,MAXI DESPENSA,"CALZADA GUATEMALA ZONA 5 BO. SANTA FE, POPTUN,...",18268-1-2024,"Supermercados, despensas y almacenes en cadena",...,350049,54461,391,Velas y candelas,NaN,None,NaN,,NaN,


___________________________________________________________________________________________

In [21]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [22]:
# Creamos un dataframe con las columnas que se van a utilizar
# Selecciona las columnas que se van a utilizar
bol = boletas[boletas['tp_poll_impute_status'].isin([351821, 351824])]
bol = boletas[boletas['detail_tp_poll_status'].isin([351634, 351635])]
bol = boletas[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']]
bol.insert(1, 'cod_prod', 
               np.where(bol['codigo_articulo'].str.len() > 8, 
                        bol['codigo_articulo'].str.slice(0, -2), 
                        bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = bol['cantidad_anterior'].astype(float)
bol['cantidad_actual'] = bol['cantidad_actual'].astype(float)
bol['precio_anterior'] = bol['precio_anterior'].astype(float)
bol['precio_actual'] = bol['precio_actual'].astype(float)
#boletas['cantidad_anterior'] = boletas.apply(lambda row: row['cantidad_actual'] if row['cantidad_anterior'] == 0 or pd.isnull(row['cantidad_anterior']) else row['cantidad_anterior'], axis=1)
#boletas['precio_anterior'] = boletas.apply(lambda row: row['precio_actual'] if row['precio_anterior'] == 0 or pd.isnull(row['precio_anterior']) else row['precio_anterior'], axis=1)

#bol

C:\Users\sncap\AppData\Local\Temp\ipykernel_36824\2388987865.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
C:\Users\sncap\AppData\Local\Temp\ipykernel_36824\2388987865.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bol['cantidad_anterior'] = bol['cantidad_anterior'].astype(float)
C:\Users\sncap\AppData\Local\Temp\ipykernel_36824\2388987865.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [23]:
#boletas['cantidad_anterior'].isnull().sum()
#bol['cantidad_actual'].isnull().sum()


#bol['cantidad_anterior'].dtype
#bol['cantidad_actual'].dtype
bol['precio_anterior'].dtype
#bol['precio_actual'].dtype

dtype('float64')

In [24]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
funciones.pondcat('Producto', ponderaciones)

,region,grupo_codigo,grupo_nombre,ponderacion_region,indice_anterior
4,0,111101,Arroz,0.522367,100.054381
5,0,111102,Maíz,0.711087,99.958663
7,0,111201,Harina de trigo,0.055905,98.964196
8,0,111202,Harina de maíz,0.170136,100.390846
9,0,111203,Harina para atoles,0.143783,99.951303
...,...,...,...,...,...
6817,8,1390901,Servicios legales y financieros,0.194022,100.000000
6818,8,1390902,Servicios funerarios,0.037005,100.000000
6819,8,1390903,Servicios de registro civil,0.056273,100.000000
6820,8,1390904,Pago de impuestos de circulación de vehículo,0.146902,100.000000


Calculo del IPC

In [25]:
#funciones que luego voy a pasar al archivo que tiene todas las funciones
def variacion(x):
    x['var_prod'] = (x['precio_actual'] / x['cantidad_actual']) * (x['cantidad_anterior'] / x['precio_anterior'])
    return x

In [26]:
# Se calcula la variacion de las variedades
variacion = variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]

#bol['var_prod'].isnull().sum()

C:\Users\sncap\AppData\Local\Temp\ipykernel_36824\1487776982.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['var_prod'] = (x['precio_actual'] / x['cantidad_actual']) * (x['cantidad_anterior'] / x['precio_anterior'])
C:\Users\sncap\AppData\Local\Temp\ipykernel_36824\385769678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)


In [27]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())

relativo

,region,cod_prod,relativo
0,1,0111101,NaN
1,1,0111102,0.984308
2,1,0111201,NaN
3,1,0111202,NaN
4,1,0111203,1.025552
...,...,...,...
2648,8,1321101,1.000000
2649,8,1321104,1.000000
2650,8,1329102,1.000000
2651,8,1390901,1.000000


In [28]:
# Step 1: Calculate geometric mean by grouping region and cod_prod
relativo = (bol01.groupby(['region', 'cod_prod'])
                  .agg(relativo=('var_prod', lambda x: gmean(x)))
                  .reset_index())

# Step 2: Left join with pon_prod DataFrame
relativo = (relativo.merge(pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']], 
                           on=['region', 'cod_prod'], 
                           how='left')
                    .rename(columns={'relativo': 'geometric_mean'}))

# Step 3: Select required columns
relativo = relativo[['region', 'cod_prod', 'grupo_nombre', 'geometric_mean', 'ponderacion_region', 'indice_anterior']]

NameError: name 'bol01' is not defined